In [1]:
import numpy as np
import itertools
import sys
sys.path.insert(0, '../CClassCSemigroups/')
import CsemigroupsCPP as CPP

In [2]:
def PrecGradLex(a,b): # Check if A<B\n",
    sumA = np.array(a).sum()
    sumB = np.array(b).sum()
    if sumA < sumB:
        return True
    elif sumA > sumB:
        return False
    else:
        c = np.array(b)-np.array(a)
        for x in c:
            if x > 0:
                return True
            if x < 0:
             return False

In [3]:
def MaximumOrder(v,order):
    maximum = v[0]
    for x in v:
        if order(maximum,x):
            maximum = x
    return maximum

In [48]:
def GetSemigroupFromGaps(gaps,eq,order): # Note: Last gap is the frobenius element.
    
    frob = list(MaximumOrder(gaps,order)) # El list es porque si no, se contamina 'gaps' por algún motivo

    mfrob = max(frob)
    
    for i in range(len(frob)):
        if frob[i] == 0:
            frob[i] = mfrob # Arreglo temporal!!!
    
    bound =  [5*mfrob for i in range(len(frob))]### <--- COMPROBAR SI ESTA COTA SE PUEDE MEJORAR!!!
    cone = []
    it = itertools.product(*[range(i+1) for i in bound])
    for x in it:
        if CPP.belongCone(list(x),eq) and not CPP.allZero(list(x)):
            cone.append(list(x))
    #print("Cone = ",cone)
    #print("gaps =",gaps)
    for x in gaps:
        cone.remove(x)
    cs = CPP.Csemigroup(cone)
    return cs

In [58]:
def ComputeCsemigroupsFixedFrobenius(cone, F,order):
    genCone = cone.GetGenerators()
    genEq = cone.GetHyperplanes()
    if CPP.allZero(F):
        raise Exception("F cannot be zero")
    genF = [x for x in genCone if not order(F,x)]
    T = [[genCone,genF]]
    G = []
    while T != []:
        print("T = ",T)
        llp = [x for x in T[0]] # Copia para evitar modificaciones
        #print("llp = ",llp)
        L = llp[0]
        D = [x for x in llp[1]]
        Lp = [x for x in D] #Copia para que no se modifique
        #print("***D =",D)
        while D != []:
            s = D[0]
            #print("s =",s)
            aux = CPP.Csemigroup(L)
            gaps = aux.GetGaps()
            gaps = gaps+[s]
            #print("gaps =",gaps)
            Lpp = GetSemigroupFromGaps(gaps,genEq,order).GetGenerators()
            #print("Lpp = ",Lpp)
            pi = [x[0] for x in T]
            #print("pi =",pi)
            if Lpp not in pi:
                if s == F:
                    G.append(Lpp)
                Lppf = [x for x in Lpp if not order(F,x)]
                #print("Lppf =", Lppf)
                if Lppf != []:
                    T.append([Lpp,Lppf])
            D.remove(s)
            #print("D =",D)
            #print("T =",T)
            if D == []:
                break
            #D = []
        #print("[L,Lp] = ",[L,Lp])
        T.remove([L,Lp])
        #print("\n")
        #T = []
    return G

In [59]:
#eq = [[0, 1], [2, -1]]
gen = [[1,0],[1,1],[1,2]]
F = [2,1]
cone = CPP.Csemigroup(gen)
ComputeCsemigroupsFixedFrobenius(cone, F, PrecGradLex)

T =  [[[[1, 0], [1, 1], [1, 2]], [[1, 0], [1, 1], [1, 2]]]]
T =  [[[[2, 0], [3, 0], [1, 1], [2, 1], [1, 2]], [[2, 0], [1, 1], [2, 1], [1, 2]]], [[[1, 0], [2, 1], [1, 2], [2, 3]], [[1, 0], [2, 1], [1, 2]]], [[[1, 0], [1, 1], [2, 3], [2, 4], [3, 6]], [[1, 0], [1, 1]]]]
T =  [[[[1, 0], [2, 1], [1, 2], [2, 3]], [[1, 0], [2, 1], [1, 2]]], [[[1, 0], [1, 1], [2, 3], [2, 4], [3, 6]], [[1, 0], [1, 1]]], [[[3, 0], [4, 0], [5, 0], [1, 1], [2, 1], [3, 1], [1, 2]], [[1, 1], [2, 1], [1, 2]]], [[[2, 0], [3, 0], [2, 1], [3, 1], [1, 2], [2, 2], [2, 3]], [[2, 0], [2, 1], [1, 2]]], [[[2, 0], [3, 0], [1, 1], [1, 2]], [[2, 0], [1, 1], [1, 2]]], [[[2, 0], [3, 0], [1, 1], [2, 1], [2, 3], [2, 4], [3, 6]], [[2, 0], [1, 1], [2, 1]]]]
T =  [[[[1, 0], [1, 1], [2, 3], [2, 4], [3, 6]], [[1, 0], [1, 1]]], [[[3, 0], [4, 0], [5, 0], [1, 1], [2, 1], [3, 1], [1, 2]], [[1, 1], [2, 1], [1, 2]]], [[[2, 0], [3, 0], [2, 1], [3, 1], [1, 2], [2, 2], [2, 3]], [[2, 0], [2, 1], [1, 2]]], [[[2, 0], [3, 0], [1, 1], [1, 2]], [[2, 0]

[[[2, 0], [3, 0], [1, 1], [1, 2]],
 [[1, 0], [3, 1], [1, 2], [2, 3]],
 [[3, 0], [4, 0], [5, 0], [1, 1], [3, 1], [1, 2], [3, 2]],
 [[2, 0], [3, 0], [3, 1], [4, 1], [1, 2], [2, 2], [2, 3], [3, 3]],
 [[3, 0],
  [4, 0],
  [5, 0],
  [3, 1],
  [4, 1],
  [5, 1],
  [1, 2],
  [2, 2],
  [3, 2],
  [2, 3],
  [3, 3]],
 [[3, 0],
  [4, 0],
  [5, 0],
  [3, 1],
  [4, 1],
  [5, 1],
  [2, 2],
  [3, 2],
  [4, 2],
  [2, 3],
  [3, 3],
  [4, 3],
  [2, 4],
  [3, 4],
  [3, 5],
  [3, 6]]]